## Setup

In [ ]:
import sys
import os
sys.path.append(os.path.join("..", ".."))
import model
import numpy as np
import matplotlib.pyplot as plt

## Data Generation

In [8]:
sim = model.sim.simulate(shapefile=os.path.join("..", "..", "data", "ba_comunas"), N0=1000, T=200, 
                         N_infected=1, step_fraction=40, distance_cutoff=0.006, seed=38)

Calculating positions...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.9min


Calculated positions in 145.87 seconds.


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  2.4min finished


Average daily contacts: 5.791666666666667
Calculating states...
t = 0; 1 infectious; 4 exposed; 999 susceptible; 0 dead
t = 20; 6 infectious; 17 exposed; 977 susceptible; 1 dead
t = 40; 12 infectious; 50 exposed; 948 susceptible; 1 dead
t = 60; 11 infectious; 56 exposed; 918 susceptible; 1 dead
t = 80; 28 infectious; 112 exposed; 853 susceptible; 2 dead
t = 100; 51 infectious; 206 exposed; 716 susceptible; 3 dead
t = 120; 47 infectious; 191 exposed; 571 susceptible; 4 dead
t = 140; 49 infectious; 188 exposed; 435 susceptible; 5 dead
t = 160; 29 infectious; 125 exposed; 344 susceptible; 5 dead
t = 180; 12 infectious; 70 exposed; 304 susceptible; 7 dead
Calculated states in 4.58 seconds.


In [ ]:
model.sim.plot_sim(sim, patients = np.arange(10), color='known_status')
plt.savefig("sample_known_status.png")

In [ ]:
model.sim.plot_sim(sim, patients = np.arange(10), color='state')
plt.savefig("sample_true_state.png")

In [ ]:
model.sim.plot_sim(sim, patients = np.arange(10), color='patient')
plt.savefig("sample_patient_id.png")

In [ ]:
model.sim.plot_infections(sim)
plt.savefig("infection_rate.png")

In [ ]:
model.sim.plot_heatmap(sim)
plt.savefig("infectious_population_density.png")

In [15]:
sim['location'].head()

,patient,date,latitude,longitude
0,0,0,-58.435613,-34.545653
1,0,1,-58.437731,-34.549600
2,0,2,-58.435019,-34.548340
3,0,3,-58.441275,-34.547083
4,0,4,-58.438393,-34.546874


In [16]:
sim['tests'].head()

,patient,date,result
0,0,0,False
1,74,0,False
2,311,0,False
3,455,0,False
0,0,1,False


In [17]:
sim['hospital'].head()

,patient,date
19,509,9
0,0,28
43,928,40
18,504,72
33,769,73


In [18]:
sim['deaths'].head()

,patient,date
4,509,10
3,504,80
7,928,88
0,127,101
6,904,121


In [22]:
import pickle
with open("simulation.pkl", 'wb') as handle:
    pickle.dump(sim, handle, protocol=4)

## Disease spread modeling

For model specification, see the [README](../../README.md)

In [23]:
%load_ext autoreload
%autoreload 2

In [24]:
# set the initial state -- at present everyone is probably healthy
state = model.model.initial_state(sim)
state

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# evaluate contacts each day and update the model
for date in sim["dates"]["date"]:
    state = model.model.next_state(sim, state, date, distance_cutoff=0.006)

In [31]:
# print the final model state
with np.printoptions(threshold=np.inf):
    print(state.round(3))

[[0.    0.    0.    0.    0.    0.    0.024 0.781 0.195]
 [0.174 0.064 0.013 0.005 0.015 0.001 0.005 0.715 0.008]
 [0.301 0.077 0.016 0.002 0.023 0.002 0.005 0.567 0.006]
 [0.308 0.052 0.013 0.003 0.017 0.002 0.006 0.593 0.006]
 [0.275 0.04  0.012 0.002 0.017 0.001 0.006 0.64  0.007]
 [0.    0.    0.    0.    0.9   0.1   0.    0.    0.   ]
 [0.    0.    0.    0.    0.076 0.005 0.042 0.866 0.011]
 [0.439 0.001 0.009 0.002 0.012 0.001 0.004 0.526 0.006]
 [0.    0.    0.    0.    0.9   0.1   0.    0.    0.   ]
 [0.    0.    0.    0.    0.357 0.032 0.054 0.554 0.003]
 [0.    0.    0.    0.    0.056 0.003 0.038 0.891 0.012]
 [0.337 0.069 0.019 0.003 0.027 0.002 0.006 0.532 0.005]
 [0.    0.    0.    0.    0.306 0.026 0.056 0.609 0.004]
 [0.    0.    0.    0.    0.486 0.047 0.047 0.42  0.001]
 [0.286 0.069 0.015 0.003 0.021 0.002 0.005 0.593 0.006]
 [0.    0.    0.    0.    0.771 0.083 0.017 0.129 0.   ]
 [0.348 0.078 0.01  0.002 0.014 0.001 0.005 0.536 0.006]
 [0.    0.    0.    0.    0.225

In [32]:
np.argmax(state, axis=1)

array([7, 7, 7, 7, 7, 4, 7, 7, 4, 7, 7, 7, 7, 4, 7, 4, 7, 7, 7, 7, 4, 7,
       7, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 7, 7, 7, 7, 7, 7,
       4, 7, 7, 7, 7, 4, 7, 7, 7, 7, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 0, 7, 7, 7, 4, 4, 7, 4, 4, 7, 7, 4, 7, 7, 7, 4,
       7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 7, 4, 7, 4, 7, 4, 7, 4, 7, 7, 7, 7,
       7, 7, 7, 4, 7, 7, 4, 4, 7, 7, 7, 7, 7, 4, 7, 7, 4, 4, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 4, 7, 7, 4, 7, 7,
       4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 0, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 7, 7, 7, 7, 4, 7, 7, 7, 4,
       7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4,
       7, 7, 7, 7, 7, 0, 7, 7, 7, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 4, 7, 7, 7, 7, 7, 4, 7, 7, 7, 7, 7, 4, 7, 7, 7, 7,
       7, 7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 4, 4, 4, 7, 4, 7, 7, 7, 7,
       7, 7, 7, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4,

In [33]:
sim['states'][:,-1]

array([7., 7., 7., 7., 0., 7., 7., 7., 0., 7., 7., 7., 7., 7., 7., 7., 7.,
       0., 0., 7., 7., 7., 7., 0., 7., 0., 7., 0., 7., 7., 7., 7., 7., 0.,
       7., 7., 7., 7., 7., 7., 7., 7., 7., 0., 7., 0., 7., 7., 7., 0., 7.,
       5., 7., 7., 7., 7., 0., 7., 7., 7., 7., 7., 7., 7., 7., 7., 0., 7.,
       7., 7., 7., 0., 0., 7., 7., 7., 0., 0., 7., 7., 7., 0., 7., 7., 7.,
       0., 0., 7., 7., 7., 7., 0., 0., 0., 7., 7., 7., 0., 7., 7., 4., 7.,
       7., 0., 7., 7., 0., 7., 7., 7., 7., 7., 0., 0., 7., 7., 7., 7., 7.,
       7., 7., 7., 7., 7., 7., 7., 7., 8., 0., 0., 8., 7., 7., 0., 7., 7.,
       0., 7., 7., 7., 0., 7., 0., 7., 7., 7., 0., 7., 7., 0., 7., 7., 0.,
       7., 7., 0., 7., 2., 7., 0., 7., 0., 0., 0., 7., 7., 0., 0., 7., 0.,
       0., 7., 7., 7., 0., 0., 7., 7., 7., 7., 7., 7., 7., 7., 0., 0., 7.,
       0., 0., 7., 0., 7., 0., 7., 0., 7., 7., 7., 7., 7., 7., 0., 0., 7.,
       7., 7., 7., 7., 7., 7., 0., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.,
       7., 7., 0., 7., 7.